In [3]:
import numpy as np
import os
import pandas as pd
from scipy import signal

## Local analysis (this recording)

In [4]:
#date_time = "2023-02-28_13-05-24"
date_time = os.path.basename(os.getcwd())
Letter = "J:"
#Letter = 'H:\Annie\Enregistrements 9-20 juin 23'
nb_E = 15    #Ajuster selon nb d'Electrodes!
ttls = [1,2]

for TTL in ttls:    #ajuster selon sites de stim
    if TTL ==1:
        brainregion = "mid"
    elif TTL ==2:
        brainregion = "vb"
    
    stim = np.load(f"{Letter}/{date_time}/Record Node 101/experiment1/recording1/events/Acquisition_Board-100.Rhythm Data/TTL/sample_numbers.npy")
    channel_states = np.load(f"{Letter}/{date_time}/Record Node 101/experiment1/recording1/events/Acquisition_Board-100.Rhythm Data/TTL/states.npy")
    cont = np.load(f"{Letter}/{date_time}/Record Node 101/experiment1/recording1/continuous/Acquisition_Board-100.Rhythm Data/sample_numbers.npy")

    EMGboolean = pd.read_pickle('EMGframeBoolean_1.pkl')
    All = np.load('RawDataChannelExtractedDS_1.npy', mmap_mode= 'r')

    EMG = All[:,0]
    y = 1

    for x in range(1, nb_E+1):
        exec(f"Ea = All[:,{y}]")
        y += 1
        exec(f"Eb = All[:,{y}]")
        y += 1
        exec(f"E{x} = Ea - Eb")

    # find the indices of the start and end times
    start_indices = np.where(channel_states == TTL)[0]
    end_indices = np.where(channel_states == -TTL)[0]

    # create a new array with the start and end times
    stim_times = np.zeros((len(start_indices), 2))
    for i in range(len(start_indices)):
        stim_times[i, 0] = stim[start_indices[i]]
        stim_times[i, 1] = stim[end_indices[i]]

    #Adjust stim timestamps - start time and sampling frequency
    start_time = cont[0]  # in 0.5 milliseconds   #First value of timestamps memmap from 'continuous' folder
    print(f'start time = {start_time} in 0.5 ms or {start_time/(2000*60)} in min')
    stim_times -= start_time
    stim_times = stim_times.astype('float64')
    stim_times /= 2.0
    stim_times = stim_times.astype('int64')






    sleepstates = ['SWS', 'REM']
    for sleepstate in sleepstates:
        w = 10.
        fs = 1000   #Sampling frequency
        freq = np.array((1.5, 2, 2.5, 3.5, 4, 4.5, 5.5, 6, 6.5, 7.5, 8, 8.5, 9.5, 10, 10.5, 11.5, 12, 12.5, 13.5, 14, 14.5, 15.5, 16, 16.5, 17.5, 18, 18.5, 19.5, 20, 20.5, 38, 40, 42, 58, 60, 62, 80, 85, 90))
        widths = w*fs / (2*freq*np.pi)

        big_table = []
        big_norm_table = []
        big_norm_table2 = []
        big_other_table = []
        all_mean_CWT_wakeremoved = []
        all_mean_CWT_wakeremoved2 = []

        #nb_E = 1
        for x in range(1,nb_E+1):
            Ele = globals()[f"E{x}"]

            E_cwt_raw = np.absolute(signal.cwt(Ele, signal.morlet2, widths, w=w))
            #for x in range(len(E_cwt_raw)):     #Ajuster pour diminution 1/f! - NON, TROP EXTREME
            #    E_cwt_raw[x] *= freq[x]
            E_cwt = np.zeros((13, len(E_cwt_raw[0])))
            y = 1
            for a in range(13):
                E_cwt[a] = np.sum(E_cwt_raw[y-1:y+1,:], axis = 0)/3
                y += 3

            BooleanWake = EMGboolean['Wake']
            CWT_wakeremoved = E_cwt.copy()
            CWT_wakeremoved = CWT_wakeremoved[:, ~BooleanWake]
            mean_CWT_wakeremoved = np.mean(CWT_wakeremoved, axis=1)
            all_mean_CWT_wakeremoved.append(mean_CWT_wakeremoved)

            CWT_wake0 = E_cwt.copy()
            
            if sleepstate == "SWS":
                BooleanState = EMGboolean['BooleanLiberal']
            elif sleepstate == "REM":
                BooleanState = EMGboolean['REMSleep']
            CWT_wake0[:, ~BooleanState] = 0
            #Compute second mean CWT only for concerned sleep state (not all sleep) #doesn't make big difference actually
            CWT_wakeremoved2 = E_cwt.copy()
            CWT_wakeremoved2 = CWT_wakeremoved2[:, BooleanState]
            mean_CWT_wakeremoved2 = np.mean(CWT_wakeremoved2, axis=1)
            all_mean_CWT_wakeremoved2.append(mean_CWT_wakeremoved2)

            #Loop through stims and collect relevant data
            ze_tableau = []
            column_titles = []
            stim_count = []
            sd_info = []
            count = 0

            all_stim_freq = (2,4,6,8,10,12,14,16,18,20,40,60,90)
            for stim_freq in all_stim_freq:
                means_allitvl = []     # groupe contient tous les itvl means
                for stim_nb in range(0, len(stim_times), 10):   #every 10th stim because blocks of 10. To be optimised.
                    stim_start = stim_times[stim_nb, 0]
                    stim_end = stim_times[stim_nb+9, 1]
                    itvl = CWT_wake0[:, stim_start:stim_end]
                    if stim_freq<30:
                        if round(9000/(stim_end - stim_start-20)) == stim_freq and 0 != np.any(itvl):
                            #print(round(9000/(stim_end - stim_start-20)))
                            mean_itvl = np.mean(itvl, axis=1)
                            means_allitvl.append(mean_itvl)
                            count +=1
                    else:
                        if round(9000/(stim_end - stim_start-20), -1) == stim_freq and 0 != np.any(itvl):
                            #print(round(9000/(stim_end - stim_start-20), -1))
                            mean_itvl = np.mean(itvl, axis=1)
                            means_allitvl.append(mean_itvl)
                            count +=1
                if len(means_allitvl) == 0:
                    #imiter ce qui est fait quand valeurs sont présentes mais mettre des NaN et 0 à la place
                    means_allitvl = np.empty((len(CWT_wake0[:,0]),1))
                    means_allitvl[:] = np.nan
                    stim_freq_mean = np.empty((len(CWT_wake0[:,0])))
                    stim_freq_mean[:] = np.nan
                    #freq_sd = np.empty((len(CWT_wake0[:,0])))
                    #freq_sd[:] = np.nan
                    nb_stims = 0
                else:
                    means_allitvl = np.transpose(np.array(means_allitvl))
                    #Aller chercher les 3 valeurs qui m'interessent pour chaque groupe de freq de stims
                    stim_freq_mean = np.mean(means_allitvl, axis=1)     # moyenne des moyennes - LA valeur importante
                    #freq_sd = np.std(means_allitvl, axis=1)
                    nb_stims = len(means_allitvl[0])
                    
                column_titles.append(f'stim: {stim_freq} Hz')
                stim_count.append(nb_stims)
                #sd_info.append(freq_sd)
                ze_tableau.append(stim_freq_mean)
                
            print(count)
            ze_tableau = pd.DataFrame(np.transpose(ze_tableau))
            ze_tableau.columns = column_titles
            row_titles = ['CWT 2 Hz', 'CWT 4 Hz', 'CWT 6 Hz', 'CWT 8 Hz', 'CWT 10 Hz', 'CWT 12 Hz', 'CWT 14 Hz', 'CWT 16 Hz', 'CWT 18 Hz', 'CWT 20 Hz', 'CWT 40 Hz', 'CWT 60 Hz', 'CWT 80 Hz',]
            ze_tableau.index = row_titles
            ze_tableau.style.set_caption(f"E{nb_E}")
            norm_table = ze_tableau.divide(mean_CWT_wakeremoved, axis=0)
            norm_table2 = ze_tableau.divide(mean_CWT_wakeremoved2, axis=0)
            lautre_tableau = [column_titles]; lautre_tableau.append(stim_count)
            big_table.append(ze_tableau)
            big_norm_table.append(norm_table)
            big_norm_table2.append(norm_table2)
            big_other_table.append(lautre_tableau)

        # Save relevant information in tables in local folder
        big_table = np.array(big_table); np.save(f'{sleepstate} Synth_table {brainregion}.npy', big_table)
        big_norm_table = np.array(big_norm_table); np.save(f'{sleepstate} Synth_norm_table {brainregion}.npy', big_norm_table)
        big_norm_table2 = np.array(big_norm_table2); np.save(f'{sleepstate} Synth_norm2_table {brainregion}.npy', big_norm_table2)
        big_other_table = np.array(big_other_table); np.save(f'{sleepstate} Otherinfo_table {brainregion}.npy', big_other_table)
        all_mean_CWT_wakeremoved = np.array(all_mean_CWT_wakeremoved); np.save(f'{sleepstate} Means_CWT_wakeremoved {brainregion}.npy', all_mean_CWT_wakeremoved)

start time = 0 in 0.5 ms or 0.0 in min
325
325
325
325
325
325
325
325
325
325
325
325
325
325
325
do nothing -> do something now
do nothing -> do something now
26
do nothing -> do something now
do nothing -> do something now
26
do nothing -> do something now
do nothing -> do something now
26
do nothing -> do something now
do nothing -> do something now
26
do nothing -> do something now
do nothing -> do something now
26
do nothing -> do something now
do nothing -> do something now
26
do nothing -> do something now
do nothing -> do something now
26
do nothing -> do something now
do nothing -> do something now
26
do nothing -> do something now
do nothing -> do something now
26
do nothing -> do something now
do nothing -> do something now
26
do nothing -> do something now
do nothing -> do something now
26
do nothing -> do something now
do nothing -> do something now
26
do nothing -> do something now
do nothing -> do something now
26
do nothing -> do something now
do nothing -> do somethin

In [ ]:
stim_times_min = stim_times/(60000)
print(stim_times_min)

In [8]:
zero = 0
nonzero =0
for i in CWT_wake0[0]:
    if i == 0:
        zero+=1
    elif i != 0:
        nonzero +=1
print(zero, nonzero)

973544 4306840


## Merge with other recordings

#### CREATE general table for stim condition

In [ ]:
mouse = 13
sleepstates = ['SWS', 'REM']
brainregions = ['midline', 'vb']

for sleepstate in sleepstates:
    for brainregion in brainregions:
        # Save in parent folder (folder for all recordings of this mouse)
        parent = os.path.dirname(os.getcwd())
        np.save(f'{parent}/{sleepstate} rawSTIMS - Mouse {mouse} - {brainregion}.npy', np.zeros((nb_E,len(CWT_wake0[:,0]),len(row_titles))))
        np.save(f'{parent}/{sleepstate} normSTIMS - Mouse {mouse} - {brainregion}.npy', np.zeros((nb_E,len(CWT_wake0[:,0]),len(row_titles))))
        np.save(f'{parent}/{sleepstate} norm2STIMS - Mouse {mouse} - {brainregion}.npy', np.zeros((nb_E,len(CWT_wake0[:,0]),len(row_titles))))
        other = np.zeros((nb_E,2,len(row_titles))); other[:,0,:] = all_stim_freq
        np.save(f'{parent}/{sleepstate} otherinfoSTIMS - Mouse {mouse} - {brainregion}.npy', other)

In [ ]:
mouse = 13
nb_E = 15
all_stim_freq = (2,4,6,8,10,12,14,16,18,20,40,60,90)
sleepstates = ['SWS', 'REM']
brainregions = ['midline', 'vb']

for sleepstate in sleepstates:
    for brainregion in brainregions:
        # Save in parent folder (folder for all recordings of this mouse)
        parent = os.path.dirname(os.getcwd())
        np.save(f'{parent}/{sleepstate} rawSTIMS - Mouse {mouse} - {brainregion}.npy', np.zeros((nb_E,13,13)))
        np.save(f'{parent}/{sleepstate} normSTIMS - Mouse {mouse} - {brainregion}.npy', np.zeros((nb_E,13,13)))
        np.save(f'{parent}/{sleepstate} norm2STIMS - Mouse {mouse} - {brainregion}.npy', np.zeros((nb_E,13,13)))
        other = np.zeros((nb_E,2,13)); other[:,0,:] = all_stim_freq
        np.save(f'{parent}/{sleepstate} otherinfoSTIMS - Mouse {mouse} - {brainregion}.npy', other)

#### UPDATE general table for stim condition

In [3]:
nb_E = 15
mouse = 13
sleepstates = ['SWS', 'REM']
brainregions = ['midline', 'vb']

for sleepstate in sleepstates:
    for brainregion in brainregions:

        #If needed: load local tables
        big_table = np.load(f'{sleepstate} Synth_table {brainregion}.npy')
        big_norm_table = np.load(f'{sleepstate} Synth_norm_table {brainregion}.npy')
        big_norm_table2 = np.load(f'{sleepstate} Synth_norm2_table {brainregion}.npy')
        big_other_table = np.load(f'{sleepstate} Otherinfo_table {brainregion}.npy', allow_pickle=True)


        parent = os.path.dirname(os.getcwd())
        rawSTIMS = np.load(f'{parent}/{sleepstate} rawSTIMS - Mouse {mouse} - {brainregion}.npy')
        normSTIMS = np.load(f'{parent}/{sleepstate} normSTIMS - Mouse {mouse} - {brainregion}.npy')
        norm2STIMS = np.load(f'{parent}/{sleepstate} norm2STIMS - Mouse {mouse} - {brainregion}.npy')
        otherinfoSTIMS = np.load(f'{parent}/{sleepstate} otherinfoSTIMS - Mouse {mouse} - {brainregion}.npy', allow_pickle=True)

        for x in range(nb_E):
            all_stim_freq = (2,4,6,8,10,12,14,16,18,20,40,60,90)
            for i, stim_freq in enumerate(all_stim_freq):
                pos_loc = i
                pos_gen = i
                stim_count_loc = int(big_other_table[x,1,pos_loc])       # nb of stims contributing to this data in local table
                stim_count_gen = otherinfoSTIMS[x,1,pos_gen]        # nb of stims contributing to this data in general table
                otherinfoSTIMS[x,1,pos_gen] = stim_count_loc + stim_count_gen       # Update nb of stims contributing to data; can be done now because same in both if/else scenarios

                if np.isnan(big_table[x,0,pos_loc]):
                    print(f"NaN at stim {stim_freq} Hz")
                else:
                    rawSTIMS[x,:,pos_gen] = (big_table[x,:,pos_loc]*stim_count_loc + rawSTIMS[x,:,pos_gen]*stim_count_gen)/(stim_count_loc + stim_count_gen)    #wheighted average
                    normSTIMS[x,:,pos_gen] = (big_norm_table[x,:,pos_loc]*stim_count_loc + normSTIMS[x,:,pos_gen]*stim_count_gen)/(stim_count_loc + stim_count_gen)    #wheighted average; but for normalised tables
                    norm2STIMS[x,:,pos_gen] = (big_norm_table2[x,:,pos_loc]*stim_count_loc + norm2STIMS[x,:,pos_gen]*stim_count_gen)/(stim_count_loc + stim_count_gen)


        # Save UPDATED VERSION in parent folder (folder for all recordings of this mouse)
        parent = os.path.dirname(os.getcwd())
        np.save(f'{parent}/{sleepstate} rawSTIMS - Mouse {mouse} - {brainregion}.npy', rawSTIMS)
        np.save(f'{parent}/{sleepstate} normSTIMS - Mouse {mouse} - {brainregion}.npy', normSTIMS)
        np.save(f'{parent}/{sleepstate} norm2STIMS - Mouse {mouse} - {brainregion}.npy', norm2STIMS)
        np.save(f'{parent}/{sleepstate} otherinfoSTIMS - Mouse {mouse} - {brainregion}.npy', otherinfoSTIMS)                